In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import datetime

In [15]:
START_POI = 0
END_POI = 405
TOTAL_DAYS = 400
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 1-(TRAIN_RATIO+VAL_RATIO)
START_DATE = '2018-12-31'
WINDOW_SIZE = 24
HORIZON = 6

In [16]:
full_dataset_path = '../full_dataset.csv'
val_prediction_path = f'../output/houston_data/train_{START_POI}_{END_POI}/predict.csv'
val_target_path = f'../output/houston_data/train_{START_POI}_{END_POI}/target.csv'
test_prediction_path = f'../output/houston_data/test_{START_POI}_{END_POI}/predict.csv'
test_target_path = f'../output/houston_data/test_{START_POI}_{END_POI}/target.csv'

In [22]:
data_frame = pd.read_csv(full_dataset_path).iloc[START_POI:END_POI]
val_predict_df = pd.read_csv(val_prediction_path, header=None)
val_target_df = pd.read_csv(val_target_path, header=None)
test_predict_df = pd.read_csv(test_prediction_path, header=None)
test_target_df = pd.read_csv(test_target_path, header=None)

In [25]:
data_frame.head(3)

,safegraph_place_id,visits_by_day,visits_by_each_hour,raw_visit_counts,location_name,street_address,city,region,postal_code,iso_country_code,...,brands,parent_safegraph_place_id,top_category,sub_category,naics_code,latitude,longitude,phone_number,open_hours,category_tags
0,sg:4cc165ff43ec4ce29e9dbe0732267ab1,[ 6428 6911 8643 8195 7936 7265 8236 61...,[ 74 42 23 ... 138 111 112],3626878,George Bush Intercontinental Airport,2800 N Terminal Rd,Houston,TX,77032.0,US,...,NaN,NaN,Support Activities for Air Transportation,Other Airport Operations,488119.0,29.981382,-95.322839,NaN,NaN,NaN
1,sg:e8af4e248bbf41549aaec725d038ee42,[2681 3062 3678 3380 3220 3136 3435 2553 2568 ...,[23 16 8 ... 40 47 47],1448988,American Express Centurion Lounge,2800 N Terminal Rd Terminal D,Houston,TX,77032.0,US,...,NaN,sg:f4a955def8ca49fd87153af82a237245,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,29.987829,-95.334916,NaN,"{ ""Mon"": [[""5:30"", ""21:00""]], ""Tue"": [[""5:30"",...",NaN
2,sg:01cebfb757224fbd8151ee6ac6b0d679,[3688 3007 3308 3478 3947 4036 2419 2122 2538 ...,[ 8 1 4 ... 19 10 7],1380483,Simon mall,5085 Westheimer Rd,Houston,TX,77056.0,US,...,Simon mall,NaN,Lessors of Real Estate,Malls,531120.0,29.738954,-95.463803,NaN,"{ ""Mon"": [[""10:00"", ""19:00""]], ""Tue"": [[""10:00...",NaN


In [26]:
val_predict_df

,0,1,2,3,4,5,6,7,8,9,...,395,396,397,398,399,400,401,402,403,404
0,69.788145,18.158104,14.021456,20.806317,16.705124,0.755315,18.755139,21.134337,9.192191,5.302088,...,0.217253,0.042776,0.585730,0.042618,0.041462,0.404797,0.043702,0.688827,0.397371,0.275845
1,48.435525,11.566983,12.684604,14.655155,11.414917,1.059516,6.574821,10.223205,3.978896,2.173234,...,0.152827,0.078384,1.069047,0.274742,0.848937,0.168368,0.033179,0.480588,0.499369,0.151833
2,63.867568,14.486942,14.742833,14.157749,23.003131,0.386935,9.062955,14.553759,7.167668,5.136776,...,0.082149,0.093798,1.079336,0.091709,0.308513,0.404347,0.222081,0.709988,0.593368,0.152848
3,88.526475,27.983976,8.672952,30.336979,33.593982,5.202599,26.864529,25.201912,11.009569,6.698751,...,0.319300,0.065172,1.191649,1.163648,0.654048,1.121700,-0.003818,0.777253,0.464699,0.191785
4,179.004083,51.298689,11.382814,40.922447,71.546206,6.623728,61.287099,53.161852,26.598312,10.697108,...,0.364086,0.119808,2.286894,3.605561,1.783008,1.866656,0.032713,0.973543,0.376157,0.127950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,308.493716,140.116818,61.393724,127.294825,105.556076,42.558259,91.849188,44.662325,44.181276,13.903392,...,1.675167,2.425091,-0.040785,-0.299121,0.018354,0.614812,1.594970,1.303256,-0.224940,2.243375
1887,320.939539,130.137550,55.277589,124.410634,119.521626,38.174461,90.198096,39.928406,49.147835,15.338772,...,1.495908,2.293694,-0.014385,-0.241844,0.186483,0.732464,1.521877,1.260315,-0.128173,2.091609
1888,343.608509,139.540129,72.244342,130.691441,114.308324,42.874218,79.994998,39.439539,56.558837,22.618626,...,1.517062,2.050506,0.122946,-0.207118,0.362029,0.840800,1.491564,1.861949,0.087231,1.710998
1889,347.416294,135.749924,75.334383,135.941976,100.108633,37.511291,67.431916,32.880975,80.770667,8.503361,...,1.349591,1.892354,0.218755,-0.168160,0.411383,0.863109,1.856981,2.020778,0.248885,2.003225


In [27]:
test_predict_df

,0,1,2,3,4,5,6,7,8,9,...,395,396,397,398,399,400,401,402,403,404
0,86.495064,27.252240,15.613608,22.525783,30.544179,1.268259,23.898563,20.722927,12.068677,5.649160,...,0.084454,0.123294,-0.105951,-0.120044,0.206514,0.226207,0.020691,0.519932,0.470629,0.319446
1,56.725514,16.780266,13.943204,13.090884,20.227384,0.190862,12.275361,13.002394,8.149891,6.706970,...,0.015126,0.019886,-0.106789,-0.073355,0.294603,0.071340,-0.040421,0.440245,0.356045,0.110396
2,57.248754,8.809763,16.900806,1.366583,26.442647,1.592257,18.178332,13.851169,9.577855,5.137802,...,0.074133,0.090165,-0.057921,-0.186000,0.157416,0.183623,0.067708,0.252894,0.452354,0.095738
3,100.796817,29.665824,19.675498,30.404522,37.803911,1.799694,29.569235,28.297725,21.297902,2.987787,...,0.217797,0.097280,-0.096012,-0.317138,0.035384,0.136370,0.188125,0.175409,0.420142,0.231114
4,210.922814,62.158882,18.960512,63.586838,79.000803,5.793508,73.849545,56.130338,41.315342,0.589268,...,0.364466,0.238846,-0.153083,-0.424005,0.157708,0.947779,0.172971,0.097336,0.229094,0.160461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,372.753313,145.204506,152.282006,136.389986,184.627349,103.076112,127.870865,83.083514,75.843795,95.725794,...,6.351203,7.586678,38.123762,14.193347,10.634436,8.065593,7.146017,4.374445,9.064460,7.217622
927,447.582497,174.329054,143.692082,166.703025,182.811554,86.230895,137.236878,71.426143,71.895969,64.869712,...,6.744639,8.742181,28.196353,14.422066,8.515729,5.554598,6.682509,4.874870,5.506270,8.893572
928,446.207296,182.275255,133.250853,167.359074,148.914527,79.512605,113.497066,63.389585,74.568889,38.161827,...,6.414167,9.581674,16.037152,11.159426,3.541264,4.091974,5.329843,8.489451,2.932879,10.730303
929,427.334100,143.997889,109.238392,120.818875,109.411376,68.107823,79.856776,50.365259,69.487557,31.769287,...,4.994941,8.999374,11.143990,11.307246,2.083798,2.565268,4.893321,11.215484,1.711327,8.085143


In [18]:
date_format = r'%Y-%m-%d'
start_time = datetime.strptime(START_DATE, date_format)
end_time = start_time + timedelta(days=TOTAL_DAYS)
start_time, end_time

In [11]:
val_date_index = date_index = pd.date_range(start=start_timestamp, periods=val_predict_df.shape[0], freq='1h')